# Personal Learning Coach with Azure Monitor Tracing
This example demonstrates a basic Azure Agents service with Azure Monitor tracing enabled.
View the results in the *Tracing* tab in your Azure AI Foundry project page.



## Imports and Setup
First, import the necessary libraries and do some setup

In [4]:
import os
import time 
import pathlib
import re

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MessageTextContent
from azure.core.settings import settings
from azure.ai.inference.tracing import AIInferenceInstrumentor 
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.core.exceptions import ClientAuthenticationError, ServiceRequestError
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.trace import get_tracer

In [5]:
# Load environment variables from .env file
# Look for .env in the current directory and parent directory
current_dir = pathlib.Path().absolute()
root_dir = current_dir.parent
load_dotenv(dotenv_path=root_dir / ".env")

# Get the project connection string from environment variables
project_connection_string = os.getenv("AZURE_AI_FOUNDRY_PROJECT_AGENTS_CONNECTION_STRING")
if not project_connection_string:
    raise ValueError("Please set AZURE_AI_FOUNDRY_PROJECT_CONNECTION_STRING in your .env file")

In [6]:
# Instrument AI Inference API
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"
settings.tracing_implementation = "opentelemetry"
AIInferenceInstrumentor().instrument()
tracer = get_tracer(__name__)

## Connect to the AI Foundry project
Now get credentials to authenticate and get handle to the AI Foundry project

In [7]:
try:
    credential = DefaultAzureCredential()
    print("✓ Successfully initialized DefaultAzureCredential")
except Exception as e:
    print(f"× Error initializing credentials: {str(e)}")

✓ Successfully initialized DefaultAzureCredential


In [8]:
try:
    project_client = AIProjectClient.from_connection_string(
        conn_str=project_connection_string,
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing project client: {str(e)}")

✓ Successfully initialized AIProjectClient


In [10]:
application_insights_connection_string = project_client.telemetry.get_connection_string()
if application_insights_connection_string:
    configure_azure_monitor(connection_string=application_insights_connection_string)
else:
    print(f"❌ No application insights configured, telemetry will not be logged to project.")


## Creating a personal learning coach agent
Lets create a specialized learning coach focused on holistic self-development and intellectual growth. 
This agent will guide users through personalized learning journeys spanning knowledge acquisition, metacognitive skills, emotional intelligence, and cognitive development.

In [11]:
def create_learning_agent():
    """Create a personal learning agent with disclaimers and basic instructions."""
    try:
        model_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")

        agent = project_client.agents.create_agent(
            model=model_name,
            name="personal-learning-coach",
            # Define the agent's behavior and responsibilities
            instructions="""
            You are a supportive Personal Learning Coach.
            You guide users through personalized learning journeys spanning knowledge acquisition, metacognitive skills, emotional intelligence, and cognitive development, but always:
            1. Include clear disclaimers about your limitations as an AI system.
            2. Encourage the user to verify important information through credible sources.
            3. Provide general learning strategies, self-reflection techniques, and knowledge-building approaches.
            4. Clearly remind them you're not a licensed educational or psychological professional.
            5. Encourage balanced approaches to learning that respect both intellectual development and mental wellbeing.
            """
        )

        print(f"🎉 Created personal learning coach, ID: {agent.id}")
        return agent
    except Exception as e:
        # Handle any errors during agent creation
        print(f"❌ Error creating agent: {str(e)}")
        return None


learning_agent = create_learning_agent()

🎉 Created personal learning coach, ID: asst_xzNAeIihAYTl7TX47bvquRxE


## Managing the conversation
A conversation (or *thread*) is where we'll store the user's messages and the agent's responses about health topics.


In [12]:
def start_conversation():
    """Create a new thread for the discussion"""
    try:
        # Create a new empty thread using the AI Foundry project client
        # A thread stores the back-and-forth messages between user and agent
        thread = project_client.agents.create_thread()
        print(f"Created a new conversation thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Error creating thread: {str(e)}")
        return None

# Initialize a new conversation thread that we'll use for our health Q&A
learning_thread = start_conversation()

Created a new conversation thread, ID: thread_4dbgVE1q3ZwlHavRHoFZXMlt


In [ ]:
def ask_question(thread_id, user_question):
    """Add a user message to the conversation thread.
    
    Args:
        thread_id: ID of the conversation thread
        user_question: The question from the user
        
    Returns:
        Message object if successful, None if error occurs
    """
    try:
        # Create a new message in the thread from the user's perspective
        # The role="user" indicates this is a user message (vs assistant)
        return project_client.agents.create_message(
            thread_id=thread_id,
            role="user", 
            content=user_question
        )
    except Exception as e:
        print(f"❌ Error adding user message: {e}")
        return None

def process_thread_run(thread_id, agent_id, max_retries=3, initial_retry_delay=1):
    """Ask the agent to process the thread and generate a response.
    
    Args:
        thread_id: ID of the conversation thread
        agent_id: ID of the health advisor agent
        max_retries: Maximum number of retry attempts (default 3)
        initial_retry_delay: Initial delay in seconds between retries (will increase exponentially)
        
    Returns:
        Run object if successful, None if error occurs
    """
    retry_count = 0
    retry_delay = initial_retry_delay
    
    while retry_count <= max_retries:
        try:
            # Create a new run to have the agent process the thread
            run = project_client.agents.create_run(
                thread_id=thread_id,
                agent_id=agent_id
            )

            # Poll the run status until completion or error
            # Status can be: queued, in_progress, requires_action, completed, failed
            while run.status in ["queued", "in_progress", "requires_action"]:
                time.sleep(1)
                run = project_client.agents.get_run(
                    thread_id=thread_id,
                    run_id=run.id
                )
            
            # If the run failed due to rate limiting, extract the wait time and retry
            if run.status == "failed" and hasattr(run, "last_error") and run.last_error.get("code") == "rate_limit_exceeded":
                error_message = run.last_error.get("message", "")
                # Try to extract the wait time from the error message
                wait_seconds = 0

                time_match = re.search(r'Try again in (\d+) seconds', error_message)
                if time_match:
                    wait_seconds = int(time_match.group(1))
                else:
                    # If unable to extract suggested extract time, use exponential backoff
                    wait_seconds = retry_delay
                    retry_delay *= 2  # Exponential backoff
                
                retry_count += 1
                if retry_count <= max_retries:
                    print(f"⏳ Rate limit exceeded. Waiting for {wait_seconds} seconds before retry ({retry_count}/{max_retries})...")
                    time.sleep(wait_seconds)
                    continue
                else:
                    print(f"❌ Rate limit exceeded. Maximum retries ({max_retries}) reached.")
                    return None
            
            # If we got here and status is not "completed", something else went wrong
            if run.status != "completed":
                print(f"🤖 Run completed with status: {run.status}")
                print(f"Error details: {run.last_error if hasattr(run, 'last_error') else 'Unknown error'}")
                return None
                
            print(f"🤖 Run completed successfully with status: {run.status}")
            return run
            
        except Exception as e:
            print(f"❌ Error processing thread run: {str(e)}")
            retry_count += 1
            if retry_count <= max_retries:
                print(f"Retrying in {retry_delay} seconds... ({retry_count}/{max_retries})")
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                print(f"Maximum retries ({max_retries}) reached.")
                return None
    
    return None

def view_thread_messages(thread_id):
    """Display all messages in the conversation thread in chronological order.
    
    Args:
        thread_id: ID of the conversation thread to display
    """
    try:
        # Get all messages in the thread
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🗣️ Conversation so far (oldest to newest):")
        
        # Loop through messages in reverse order to show oldest first
        for m in reversed(messages.data):
            if m.content:
                # Extract the text content from the message. We only handle text messages
                last_content = m.content[-1]
                if isinstance(last_content, MessageTextContent):
                    print(f"{m.role.upper()}: {last_content.text.value}\n")
        print("-----------------------------------\n")
    except Exception as e:
        print(f"❌ Error viewing thread: {str(e)}")

### Example Queries
Let's do some quick queries now to see the agent's disclaimers and how it handles typical health questions. We'll ask about **BMI** and about **balanced meal** for an active lifestyle.


In [14]:
# First verify that we have valid agent and thread objects before proceeding
if learning_agent and learning_thread:

    msg = ask_question(learning_thread.id, "How can I improve my reading comprehension and retention?")
    run = process_thread_run(learning_thread.id, learning_agent.id)

    msg = ask_question(learning_thread.id, "What questions should I ask myself when reflecting on my learning progress?")
    run = process_thread_run(learning_thread.id, learning_agent.id)

    view_thread_messages(learning_thread.id)
else:
    print("❌ Could not run example queries because agent or thread is None.")

🤖 Run completed successfully with status: RunStatus.COMPLETED
⏳ Rate limit exceeded. Waiting for 43 seconds before retry (1/3)...
🤖 Run completed successfully with status: RunStatus.COMPLETED

🗣️ Conversation so far (oldest to newest):
USER: How can I improve my reading comprehension and retention?

ASSISTANT: Improving reading comprehension and retention is a fantastic goal, and I'm here to support you on this learning journey! Here are some strategies that might help:

1. **Active Reading Techniques**:
   - **Preview the Material**: Before diving into the text, skim headings, subheadings, and any highlighted or bolded text to get a sense of the main ideas.
   - **Annotate as You Read**: Take notes in the margins, highlight key points, or underline important concepts. This engagement helps reinforce understanding and improves retention.

2. **Summarization**:
   - After reading a section, pause to summarize what you've just read in your own words. This will help reinforce the material

## Cleanup
If preferred, you can remove the agent from the AI Foundry service. In production, you might keep your agent to preserve complete observability.

In [ ]:
def cleanup(agent):
    if agent:
        try:
            project_client.agents.delete_agent(agent.id)
            print(f"🗑️ Deleted agent: {agent.name}")
        except Exception as e:
            print(f"Error cleaning up agent: {e}")
    else:
        print("No agent to clean up.")

# Call cleanup function to delete our agent
cleanup_active = False

if cleanup_active:
    cleanup(learning_agent)